In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import modin.pandas as pd
from fastai.tabular import *
from utils import isVaildDate, is_more

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-07_17-10-44_18901/logs.
Waiting for redis server at 127.0.0.1:16951 to respond...
Waiting for redis server at 127.0.0.1:14856 to respond...
Starting Redis shard with 10.0 GB max memory.
Starting the Plasma object store with 10.0 GB memory using /tmp.


## 数据清洗

In [3]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user'),
 PosixPath('../test_A/imps_log')]

In [4]:
u_data = root/'user'
u_data.ls()
log_data = root/'imps_log'
log_data.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

- 广告静态数据读入以备用

In [5]:
ad_static = pd.read_csv('../data/ad_static.csv', low_memory=False, encoding='utf-8')

In [6]:
ad_static.shape
ad_static.tail()

(502153, 7)

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
502148,82588,2019-03-19 03:56:04,21666,21917,18,198,64
502149,665036,2019-02-21 09:08:35,6184,11633,18,232,64
502150,491914,2019-01-25 05:01:10,1809,5278,5,232,64
502151,3990,2019-03-01 08:17:08,4255,3401,18,218,36
502152,222150,2019-02-28 07:56:53,4250,14527,18,94,36


### 历史曝光日志数据文件

In [7]:
col_names = ['广告请求id', '广告请求时间', '广告位id', '用户id', '曝光广告id', '曝光广告素材尺寸', '曝光广告出价bid',
             '曝光广告pctr', '曝光广告quality_ecpm', '曝光广告totalEcpm']

In [8]:
c_sz = 10240

In [ ]:
logs = pd.read_csv(log_data/'totalExposureLog.out', sep='\t', header=None, names=col_names, chunksize=c_sz)

- 看下文件信息

In [ ]:
!du -h {log_data}/'totalExposureLog.out'

6.2G	../test_A/imps_log/totalExposureLog.out


In [ ]:
!cat {log_data}/'totalExposureLog.out' | wc -l

102386695


6.2G 数据 1亿多行数据

- 删除空值 数据量大 考虑删除而不是插值，因为我们不缺数据

In [ ]:
def invalid_date(df_row, field='广告请求时间'):
    """是否删除当前行,首先转为时间格式之后再行本操作"""
    if not isVaildDate(str(df_row[field])):
        df_row[field] = np.nan
    
    return df_row

- 获取有效广告 id

In [ ]:
ad_id = ad_static['广告id'].unique()

In [ ]:
ad_id.shape

(502153,)

In [ ]:
def invalid_resquest_ad_id(df_row, field=None):
    if df_row[field] in ad_id:
        return df_row

- 删除非法多值

In [ ]:
def invalid_more_value(df_row, field=None):
    if len(str(df_row[field]).split(',')) == 1:
        return df_row

- 首先应该删除 `'../data/ad_logs.csv'` 因为接下来我们是以 append 方式写文件

In [ ]:
fn = '../data/ad_logs.csv'
if os.path.exists(fn):
    os.remove(fn)

In [ ]:
?partial

- 生成器 读写同时进行（备份）

In [ ]:
for df in logs:
    # 3. 去掉非法时间行
    df['广告请求时间'] = pd.to_datetime(df['广告请求时间'], unit='s')  # 转为日期
    df = df.apply(invalid_date, axis=1)
    # 1. 去空值
    df = pd.merge(df, ad_static, left_on='曝光广告id', right_on='广告id', how='inner')
    df.dropna(axis=0, how='any', inplace=True)
    # 2. 去重 所有列相同
    df.drop_duplicates(subset=None, keep='first', inplace=True)
    # 4. 数据类型转换
    # 暂无
    df.dropna(axis=0, how='any', inplace=True)
    df.reset_index(drop=True, inplace=True)  # 为了正常访问，重建索引
    print(df.shape)
    df.to_csv('../data/ad_logs.csv', mode='a', index=None, encoding='utf-8', header=False)

(10044, 17)
(10048, 17)
(10045, 17)
(10023, 17)
(10039, 17)
(10036, 17)
(10058, 17)
(10042, 17)
(10055, 17)
(10076, 17)
(10066, 17)
(10046, 17)
(10063, 17)
(10061, 17)
(10063, 17)
(10054, 17)
(10089, 17)
(10032, 17)
(10043, 17)
(10068, 17)
(10062, 17)
(10056, 17)
(10054, 17)
(10072, 17)
(10076, 17)
(10036, 17)
(10054, 17)
(10031, 17)
(10048, 17)
(10087, 17)
(10028, 17)
(10073, 17)
(10016, 17)
(10060, 17)
(10036, 17)
(10058, 17)
(10082, 17)
(10053, 17)
(10037, 17)
(10055, 17)
(10053, 17)
(10053, 17)
(10073, 17)
(10067, 17)
(10022, 17)
(10064, 17)
(10066, 17)
(10073, 17)
(10034, 17)
(10051, 17)
(10067, 17)
(10058, 17)
(10050, 17)
(10038, 17)
(10037, 17)
(10058, 17)
(10077, 17)
(10050, 17)
(10044, 17)
(10064, 17)
(10067, 17)
(10063, 17)
(10048, 17)
(10080, 17)
(10065, 17)
(10027, 17)
(10062, 17)
(10051, 17)
(10065, 17)
(10027, 17)
(10063, 17)
(10066, 17)
(10080, 17)
(10053, 17)
(10058, 17)
(10046, 17)
(10056, 17)
(10021, 17)
(10058, 17)
(10075, 17)
(10043, 17)
(10063, 17)
(10065, 17)
(100

- 重新读入 全部数据

In [ ]:
logs1 = pd.read_csv('../data/ad_logs.csv', sep='\t', header=None, names=col_names+list(ad_static.columns))

 - 再次去重

In [ ]:
logs1.drop_duplicates(subset=None, keep='first', inplace=True)

In [ ]:
logs1.shape

In [ ]:
logs1.to_csv('../data/ad_logs.csv', mode='w', index=None, encoding='utf-8')

- 看下文件信息

In [ ]:
!du -h {fn}

In [ ]:
!cat {fn} | wc -l